In [1]:
with open("shakespeare.txt", "r", encoding="utf-8") as f:
    data = f.read()

f"dataset length {len(data)}"

'dataset length 1115394'

In [2]:
chars = sorted(list(set(data)))
vocab_size = len(chars)
f"vocab size: {vocab_size}; vocab: {''.join(chars)}"

"vocab size: 65; vocab: \n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [3]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [4]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [ ]:
# Self attention head

B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False) # Kinda the metadata contained by each token (how relevant each token is), bias=False - no added constant :)
query = nn.Linear(C, head_size, bias=False) # Kinda the information each token is looking for
value = nn.Linear(C, head_size, bias=False) # The actual data contained at each token
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
# Similarity between each key and query is the weights
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)
# Weights are now data dependent

tril = torch.tril(torch.ones(T, T)) # (T, T)
# Mask future tokens
wei = wei.masked_fill(tril==0, float("-inf")) # (B, T, T)
wei = F.softmax(wei, dim=-1)

# Value tensor, the actual data contained by each token, what is offered by each token
v = value(x) # (B, T, 16)

# The Query-Key mechanism decides how much of each value should contribute to the final representation of a token.
out = wei @ v # (B, T, T) @ (B, T, 16) --> (B, T, 16)
out.shape

torch.Size([4, 8, 32])

Notes:
- Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
-"self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        key = nn.Linear(C, head_size, bias=False)
        query = nn.Linear(C, head_size, bias=False)
        value = nn.Linear(C, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))